# Combine Input data - Operational Congestion Management
Converts raw data exports from the DataLake into a prepped input format.

The raw files are not included in the repository.

This notebook can be used as reference on how to prep data from your own DataLake, and for inspection of some intermediate data.

In [2]:
import glob
import warnings
import datetime
import os
import yaml

import numpy as np
import pandas as pd

import scipy

from tqdm import tqdm_notebook as tqdm

import cufflinks
cufflinks.go_offline()
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
pio.renderers.default = 'iframe'

from openstef.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test
from openstef.metrics.figure import plot_feature_importance
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass

2022-11-29 14:55.14 [info     ] Proloaf not available, setting constructor to None


## Retrieve other input data
First things first, lets prep all the relevant input data:
 - measurements of windpark
 - measurements of nearby solar park
 - realised_curtailment (from actual scaled down wind power (NOT LC_windpark estimates)

## Estimate actual curtailment (based on n_turbines curtailed)
For Hallum, mitigations are 'turn of n turbines'. The LC_windpark_Nijkerkerpolder estimates the curtailment based on windspeed and turbine specs. <br>
We can improve this by rescaling the actual generation of the turbines which are not curtailed.<br>

The measured load is corrected for mitigations which where activated. <br>
This was for example the case on April 5th 2021:<br>
![Stuurcorrecties](input/figs/Hal_202104_stuurcorrecties.PNG)


In [26]:
# Define datafiles
hal_pv_file = 'raw/hal_pv_from_lw_2020-2022.csv' # old: '../ERC_cm_data/hal_pv_from_lw.csv'
hal_load_wind_file = 'raw/hal_wind_trafo_wob_2020_2022.csv' # old: '../ERC_cm_data/hal_wind_trafo_wob.csv'
hal_mitigation_actions_file = 'raw/hal_mitigation_actions.csv'
hal_operational_forecasts_file =  'raw/435_24h_p98.csv' # '../ERC_cm_data/435_24h_p98_2020_2022.csv' # old:

# Read operational forecasts
d = pd.read_csv(hal_operational_forecasts_file, index_col=0, parse_dates=True).tz_localize('UTC')

# Load PV, PV park (LW 10-5 V5.05) -> download manually from the wop (mrid = 725e71a0-1288-5105-96b7-4876d5061845, perhaps in future retrieve from TelRegistry?
p = pd.read_csv(hal_pv_file, sep=';', decimal=',')
# Note that WOP is in local time
p['datetime'] = pd.to_datetime([f'{d}T{t}' for d, t in zip(p[' Datum'], p['Tijd'])]).tz_localize('CET', ambiguous='NaT', nonexistent='NaT').tz_convert('UTC')
# remove NAT
p = p[~p.datetime.isnull()]
p = p.set_index('datetime').iloc[:,2:-1]
hal_pv = p
hal_pv.columns=['PV - reference']

# Load wind &  measured_load -> retrieve manually from wop (Hal- V06 (trafo: 99120451-786d-5761-86c9-58f2a29013c5) and V08 (windpark: 45e4075e-0723-5adf-9c3f-b9e1bae61b4c) )
# Note that the wind_ref needs to be corrected for mitigations as well! This will be done at the same point as where the load is corrected.
h = pd.read_csv(hal_load_wind_file, sep=';', decimal=',')
# Note that WOP is in local time
h['datetime'] = pd.to_datetime([f'{d}T{t}' for d, t in zip(h[' Datum'], h['Tijd'])]).tz_localize('CET', ambiguous='NaT', nonexistent='NaT').tz_convert('UTC')
# Remove NATs
h = h[~h.datetime.isnull()]

h = h.set_index('datetime').iloc[:,2:-1]
# rename
hal_names = {'Hal-RS-10-G-V06-P':'Obs',
             'Hal-RS-10-G-V08-P':'Wind'}
h=h.rename(columns=hal_names)
# Invert polarity of obs
h['Obs'] *= -1

# Combine it all
hal = d.loc[:,['Realised - Total', '24.0 Hours Ahead']].merge(h, left_index=True, right_index=True, how='left').merge(hal_pv, left_index=True, right_index=True, how='left')

## Plot Figure 1

In [4]:
# Plot timeseries of max, avg and min load per day
dailies = d[['Realised - Total']].resample('1D').aggregate(['mean',
                                                            'max',
                                                            'min'])['Realised - Total']

fig1a = dailies.iplot(layout=dict(template='plotly_white',
                          yaxis=dict(title='Daily load [MW]'),
                          width=600, height=400), asFigure=True)
# manually hide 'mean' for clarity
fig1a.write_image("results/figs/fig1a.svg")


In [5]:
fig1a

In [6]:
fig1b = d.loc['2021-04-02':'2021-04-09','Realised - Total'].iplot(layout=dict(template='plotly_white',
                          yaxis=dict(title='Load [MW]'),
                          width=600, height=300), asFigure=True)
fig1b.write_image('results/figs/fig1b.svg')

## Correct for mitigations

In [7]:
# List all stuur actions. Received by HvdG, from MvV, used for compenstation
mit_actions = pd.read_csv(hal_mitigation_actions_file, sep='\t')
mit_actions['date']=[x.date() for x in pd.to_datetime(mit_actions['Datum'])]
indices = [pd.date_range(start=f'{x[0]} {x[3]}', end=f'{x[0]} {x[4]}', freq='15T') for _, x in mit_actions.iterrows()]
indices = [item for sublist in indices for item in sublist]
scaled_mit_df = pd.DataFrame(index=indices).tz_localize('CET')
# Merge with actions
scaled_mit_df['date'] = scaled_mit_df.index.date
scaled_mit_df = scaled_mit_df.reset_index().merge(mit_actions[['date','aantal turbines uitgezet', 'inzet']], left_on='date', right_on='date', how='outer').set_index('index')
scaled_mit_df = scaled_mit_df.merge(hal[['Wind']].dropna().rename(columns=dict(Wind='generated_wind')), left_index=True, right_index=True, how='left')
scaled_mit_df['frac_turbines_off'] = scaled_mit_df['aantal turbines uitgezet']/7.
scaled_mit_df['curtailed_power'] = scaled_mit_df['frac_turbines_off']*scaled_mit_df['generated_wind']
# curtailed_power equals LoadCorrections_Actual_wind, based on measurements of uncurtailed windturbines
scaled_mit_df 

,date,aantal turbines uitgezet,inzet,generated_wind,frac_turbines_off,curtailed_power
index,,,,,,
2020-04-05 06:00:00+00:00,2020-04-05,3,gepland,NaN,0.428571,NaN
2020-04-05 06:15:00+00:00,2020-04-05,3,gepland,NaN,0.428571,NaN
2020-04-05 06:30:00+00:00,2020-04-05,3,gepland,NaN,0.428571,NaN
2020-04-05 06:45:00+00:00,2020-04-05,3,gepland,NaN,0.428571,NaN
2020-04-05 07:00:00+00:00,2020-04-05,3,gepland,NaN,0.428571,NaN
...,...,...,...,...,...,...
2022-06-12 15:00:00+00:00,2022-06-12,2,gepland,NaN,0.285714,NaN
2022-06-12 15:15:00+00:00,2022-06-12,2,gepland,NaN,0.285714,NaN
2022-06-12 15:30:00+00:00,2022-06-12,2,gepland,NaN,0.285714,NaN


In [8]:
fig = go.Figure()
for inzet in ['gepland','ongepland']:
    subset = scaled_mit_df[scaled_mit_df.inzet==inzet]
    fig.add_scatter(x=subset.index, y=subset['aantal turbines uitgezet'], name=inzet, mode='markers')
fig.show()

In [9]:
# Compare curtailed energy 
scaled_mit_df.pivot_table(index='inzet', values='curtailed_power', aggfunc='sum')

,curtailed_power
inzet,
gepland,-168.735714
ongepland,-21.394286


In [10]:
# Count days
scaled_mit_df.pivot_table(index='inzet', values='date', aggfunc='nunique')

,date
inzet,
gepland,48
ongepland,7


In [11]:
scaled_mit_df['2021-02-27':'2022-03-30'].pivot_table(index='inzet', values='date', aggfunc='nunique')

,date
inzet,
gepland,15
ongepland,1


In [12]:
# Congestion window: march 20th - sept 7th (observed mitigations +- 1 week)

In [29]:
# Add to central dataframe
hal = hal.merge(scaled_mit_df['curtailed_power'], left_index=True, right_index=True, how='left')

In [30]:
# Correct observations for congestion mitigations !!! Pay attention to the signs !!!
hal['Uncurtailed_true'] = hal['Obs']+hal['curtailed_power'].fillna(0)
hal['Wind_uncurtailed'] = hal['Wind']-hal['curtailed_power'].fillna(0) 
hal.resample('15T').mean().iplot()

In [39]:
## Give variables clear names
clear_names = {'Uncurtailed_true':'Load_corrected_for_curtailment',
               'Wind_uncurtailed':'Wind_reference',
               'PV - reference':'PV_reference',
               '24.0 Hours Ahead':'Day_ahead_forecast', 
}

# Store relevant columns with clear names as inputs.csv
hal[[col for col in clear_names.keys()]].rename(columns=clear_names).to_csv('input/prepped_inputs.csv')


In [40]:
# Present overview of the timeseries
hal['Realised - Total'].resample('1D').aggregate([np.max, np.median, np.min])\
.iplot(layout=dict(template='plotly_white', yaxis=dict(title='Measured load [MW]'), xaxis=dict(), width=1000, height=200, margin=dict(t=0,b=0,r=0,l=0)))

In [41]:
# Prep predictive features
features = pd.read_csv('input/difficult_case_Hallum_ERC_from_2020-01-01_to_2022-08-03_test_data.csv', index_col=0, parse_dates=True) # Test data contains the relevant time period
# Note that the weather features are timeshifted one hour! Fix this here for now
weather_features = ['clouds', 'radiation', 'temp', 'winddeg', 'windspeed',
       'windspeed_100m', 'pressure', 'humidity', 'rain', 'mxlD', 'snowDepth',
       'clearSky_ulf', 'clearSky_dlf']
features[weather_features]=features[weather_features].shift(freq='H', periods=1)

# Only keep relevant time period and store as csv
features[hal.index.min():hal.index.max()].to_csv('input/predictors.csv')
